In [1]:
from itertools import permutations
from collections import deque
import heapq

def generate_single_translocation(perm, start_idx, end_idx, insert_pos):
    """
    Generate a single translocation: move interval [start_idx:end_idx] to insert_pos.
    Returns the resulting permutation.
    """
    # Extract the interval (keeping its internal order)
    interval = perm[start_idx:end_idx]
    # Get the remaining elements
    remaining = perm[:start_idx] + perm[end_idx:]
    # Insert interval at the specified position
    result = remaining[:insert_pos] + interval + remaining[insert_pos:]
    return result

def get_all_possible_translocations(perm):
    """
    Get all possible single translocations from a given permutation.
    Returns list of (result_permutation, move_info) tuples.
    """
    translocations = []
    n = len(perm)
    
    # Try all possible intervals
    for start_idx in range(n):
        for end_idx in range(start_idx + 1, n + 1):
            interval = perm[start_idx:end_idx]
            remaining = perm[:start_idx] + perm[end_idx:]
            
            # Try all possible insertion positions
            for insert_pos in range(len(remaining) + 1):
                result = remaining[:insert_pos] + interval + remaining[insert_pos:]
                
                # Only include if different from original
                if result != perm:
                    move_info = {
                        'interval': interval,
                        'from_pos': (start_idx, end_idx),
                        'to_pos': insert_pos,
                        'description': f"Move {interval} from positions [{start_idx}:{end_idx}] to position {insert_pos}"
                    }
                    translocations.append((result, move_info))
    
    return translocations

def find_all_paths_exact_length(start_perm, target_perm, exact_length):
    """
    Find ALL possible paths of exactly the specified length.
    Returns list of (path, moves) tuples.
    """
    all_paths = []
    
    # Use DFS to explore all paths
    def dfs(current_perm, current_path, current_moves, remaining_steps):
        if remaining_steps == 0:
            if current_perm == target_perm:
                all_paths.append((current_path.copy(), current_moves.copy()))
            return
        
        # Generate all possible moves from current position
        possible_moves = get_all_possible_translocations(current_perm)
        
        for next_perm, move_info in possible_moves:
            # Avoid revisiting the same permutation in the current path (cycle detection)
            if next_perm not in current_path:
                current_path.append(next_perm)
                current_moves.append(move_info)
                
                dfs(next_perm, current_path, current_moves, remaining_steps - 1)
                
                # Backtrack
                current_path.pop()
                current_moves.pop()
    
    # Start DFS
    dfs(start_perm, [start_perm], [], exact_length)
    
    return all_paths

def analyze_all_paths(start_perm, target_perm, exact_length):
    """
    Find and display all possible paths of exact length.
    """
    start_str = ''.join(map(str, start_perm))
    target_str = ''.join(map(str, target_perm))
    
    print(f"FINDING ALL PATHS: {start_str} → {target_str} in exactly {exact_length} steps")
    print("=" * 80)
    
    all_paths = find_all_paths_exact_length(start_perm, target_perm, exact_length)
    
    if not all_paths:
        print(f"No paths found of exactly {exact_length} steps!")
        return
    
    print(f"Found {len(all_paths)} different paths of exactly {exact_length} steps:")
    print()
    
    for path_num, (path, moves) in enumerate(all_paths, 1):
        print(f"PATH {path_num}:")
        print("-" * 40)
        
        for i, perm in enumerate(path):
            perm_str = ''.join(map(str, perm))
            if i == 0:
                print(f"Step {i}: {perm_str} (start)")
            else:
                print(f"Step {i}: {perm_str}")
                print(f"       ← {moves[i-1]['description']}")
        
        print()
    
    return all_paths

def find_shortest_path_length(start_perm, target_perm, max_depth=10):
    """
    Find the shortest path length using BFS.
    """
    if start_perm == target_perm:
        return 0
    
    queue = deque([(start_perm, 0)])
    visited = {start_perm}
    
    while queue:
        current_perm, depth = queue.popleft()
        
        if depth >= max_depth:
            continue
            
        moves = get_all_possible_translocations(current_perm)
        for next_perm, _ in moves:
            if next_perm == target_perm:
                return depth + 1
            
            if next_perm not in visited:
                visited.add(next_perm)
                queue.append((next_perm, depth + 1))
    
    return float('inf')

def comprehensive_analysis(start_perm, target_perm):
    """
    Comprehensive analysis: find shortest path, then show all paths of that length.
    """
    start_str = ''.join(map(str, start_perm))
    target_str = ''.join(map(str, target_perm))
    
    print(f"COMPREHENSIVE ANALYSIS: {start_str} → {target_str}")
    print("=" * 80)
    
    # First, find the shortest path length
    shortest_length = find_shortest_path_length(start_perm, target_perm)
    
    if shortest_length == float('inf'):
        print("No path found!")
        return
    
    print(f"Shortest path length: {shortest_length} steps")
    print()
    
    # Now find all paths of that length
    analyze_all_paths(start_perm, target_perm, shortest_length)
    
    # Also show paths of length shortest_length + 1 if requested
    if shortest_length < 5:  # Don't go too deep
        print(f"\nPaths of length {shortest_length + 1}:")
        print("=" * 80)
        longer_paths = find_all_paths_exact_length(start_perm, target_perm, shortest_length + 1)
        print(f"Found {len(longer_paths)} paths of length {shortest_length + 1}")

if __name__ == "__main__":
    # Main test case: 1234 → 4321
    start = (1, 2, 3, 4)
    target = (4, 3, 2, 1)
    
    print("SPECIFIC REQUEST: All possible ways to transform 1234 → 4321 in exactly 3 steps")
    print("=" * 80)
    
    # Find all 3-step paths
    all_3_step_paths = analyze_all_paths(start, target, 3)
    
    print(f"\nSUMMARY:")
    print(f"Total number of different 3-step paths: {len(all_3_step_paths) if all_3_step_paths else 0}")
    
    # Also check what the shortest path actually is
    print("\n" + "=" * 80)
    print("VERIFICATION: What is the actual shortest path length?")
    shortest = find_shortest_path_length(start, target)
    print(f"Shortest path length: {shortest}")
    
    if shortest != 3:
        print(f"\nNote: The shortest path is {shortest} steps, not 3.")
        print("Showing all optimal paths:")
        analyze_all_paths(start, target, shortest)

SPECIFIC REQUEST: All possible ways to transform 1234 → 4321 in exactly 3 steps
FINDING ALL PATHS: 1234 → 4321 in exactly 3 steps
Found 320 different paths of exactly 3 steps:

PATH 1:
----------------------------------------
Step 0: 1234 (start)
Step 1: 2134
       ← Move (1,) from positions [0:1] to position 1
Step 2: 3214
       ← Move (2, 1) from positions [0:2] to position 1
Step 3: 4321
       ← Move (3, 2, 1) from positions [0:3] to position 1

PATH 2:
----------------------------------------
Step 0: 1234 (start)
Step 1: 2134
       ← Move (1,) from positions [0:1] to position 1
Step 2: 3214
       ← Move (2, 1) from positions [0:2] to position 1
Step 3: 4321
       ← Move (4,) from positions [3:4] to position 0

PATH 3:
----------------------------------------
Step 0: 1234 (start)
Step 1: 2134
       ← Move (1,) from positions [0:1] to position 1
Step 2: 3421
       ← Move (2, 1) from positions [0:2] to position 2
Step 3: 4321
       ← Move (3,) from positions [0:1] to position

In [3]:
from itertools import permutations
from collections import deque
import heapq

def generate_single_translocation(perm, start_idx, end_idx, insert_pos):
    """
    Generate a single translocation: move interval [start_idx:end_idx] to insert_pos.
    Returns the resulting permutation.
    """
    # Extract the interval (keeping its internal order)
    interval = perm[start_idx:end_idx]
    # Get the remaining elements
    remaining = perm[:start_idx] + perm[end_idx:]
    # Insert interval at the specified position
    result = remaining[:insert_pos] + interval + remaining[insert_pos:]
    return result

def get_all_possible_translocations(perm):
    """
    Get all possible single translocations from a given permutation.
    Returns list of (result_permutation, move_info) tuples.
    """
    translocations = []
    n = len(perm)
    
    # Try all possible intervals
    for start_idx in range(n):
        for end_idx in range(start_idx + 1, n + 1):
            interval = perm[start_idx:end_idx]
            remaining = perm[:start_idx] + perm[end_idx:]
            
            # Try all possible insertion positions
            for insert_pos in range(len(remaining) + 1):
                result = remaining[:insert_pos] + interval + remaining[insert_pos:]
                
                # Only include if different from original
                if result != perm:
                    move_info = {
                        'interval': interval,
                        'from_pos': (start_idx, end_idx),
                        'to_pos': insert_pos,
                        'description': f"Move {interval} from positions [{start_idx}:{end_idx}] to position {insert_pos}"
                    }
                    translocations.append((result, move_info))
    
    return translocations

def find_all_paths_exact_length(start_perm, target_perm, exact_length):
    """
    Find ALL possible paths of exactly the specified length.
    Returns list of (path, moves) tuples.
    """
    all_paths = []
    
    # Use DFS to explore all paths
    def dfs(current_perm, current_path, current_moves, remaining_steps):
        if remaining_steps == 0:
            if current_perm == target_perm:
                all_paths.append((current_path.copy(), current_moves.copy()))
            return
        
        # Generate all possible moves from current position
        possible_moves = get_all_possible_translocations(current_perm)
        
        for next_perm, move_info in possible_moves:
            # Avoid revisiting the same permutation in the current path (cycle detection)
            if next_perm not in current_path:
                current_path.append(next_perm)
                current_moves.append(move_info)
                
                dfs(next_perm, current_path, current_moves, remaining_steps - 1)
                
                # Backtrack
                current_path.pop()
                current_moves.pop()
    
    # Start DFS
    dfs(start_perm, [start_perm], [], exact_length)
    
    return all_paths

def analyze_all_paths(start_perm, target_perm, exact_length):
    """
    Find and display all possible paths of exact length.
    """
    start_str = ''.join(map(str, start_perm))
    target_str = ''.join(map(str, target_perm))
    
    print(f"FINDING ALL PATHS: {start_str} → {target_str} in exactly {exact_length} steps")
    print("=" * 80)
    
    all_paths = find_all_paths_exact_length(start_perm, target_perm, exact_length)
    
    if not all_paths:
        print(f"No paths found of exactly {exact_length} steps!")
        return
    
    print(f"Found {len(all_paths)} different paths of exactly {exact_length} steps:")
    print()
    
    for path_num, (path, moves) in enumerate(all_paths, 1):
        print(f"PATH {path_num}:")
        print("-" * 40)
        
        for i, perm in enumerate(path):
            perm_str = ''.join(map(str, perm))
            if i == 0:
                print(f"Step {i}: {perm_str} (start)")
            else:
                print(f"Step {i}: {perm_str}")
                print(f"       ← {moves[i-1]['description']}")
        
        print()
    
    return all_paths

def find_shortest_path_length(start_perm, target_perm, max_depth=10):
    """
    Find the shortest path length using BFS.
    """
    if start_perm == target_perm:
        return 0
    
    queue = deque([(start_perm, 0)])
    visited = {start_perm}
    
    while queue:
        current_perm, depth = queue.popleft()
        
        if depth >= max_depth:
            continue
            
        moves = get_all_possible_translocations(current_perm)
        for next_perm, _ in moves:
            if next_perm == target_perm:
                return depth + 1
            
            if next_perm not in visited:
                visited.add(next_perm)
                queue.append((next_perm, depth + 1))
    
    return float('inf')

def comprehensive_analysis(start_perm, target_perm):
    """
    Comprehensive analysis: find shortest path, then show all paths of that length.
    """
    start_str = ''.join(map(str, start_perm))
    target_str = ''.join(map(str, target_perm))
    
    print(f"COMPREHENSIVE ANALYSIS: {start_str} → {target_str}")
    print("=" * 80)
    
    # First, find the shortest path length
    shortest_length = find_shortest_path_length(start_perm, target_perm)
    
    if shortest_length == float('inf'):
        print("No path found!")
        return
    
    print(f"Shortest path length: {shortest_length} steps")
    print()
    
    # Now find all paths of that length
    analyze_all_paths(start_perm, target_perm, shortest_length)
    
    # Also show paths of length shortest_length + 1 if requested
    if shortest_length < 5:  # Don't go too deep
        print(f"\nPaths of length {shortest_length + 1}:")
        print("=" * 80)
        longer_paths = find_all_paths_exact_length(start_perm, target_perm, shortest_length + 1)
        print(f"Found {len(longer_paths)} paths of length {shortest_length + 1}")

def analyze_step_by_step_translocations(start_perm, target_perm):
    """
    Analyze all possible first and second translocations systematically.
    """
    start_str = ''.join(map(str, start_perm))
    target_str = ''.join(map(str, target_perm))
    
    print(f"STEP-BY-STEP ANALYSIS: {start_str} → {target_str}")
    print("=" * 80)
    
    # Get all possible first translocations
    first_moves = get_all_possible_translocations(start_perm)
    
    print(f"ALL POSSIBLE FIRST TRANSLOCATIONS from {start_str}:")
    print("-" * 60)
    
    first_results = {}  # Store results for second step analysis
    
    for i, (result_perm, move_info) in enumerate(first_moves, 1):
        result_str = ''.join(map(str, result_perm))
        print(f"{i:2d}. {start_str} → {result_str}")
        print(f"    Operation: {move_info['description']}")
        
        # Group by result for second step analysis
        if result_perm not in first_results:
            first_results[result_perm] = []
        first_results[result_perm].append((i, move_info))
        print()
    
    print(f"Total: {len(first_moves)} possible first translocations")
    print(f"Leading to {len(first_results)} unique intermediate permutations")
    
    print("\n" + "=" * 80)
    print("ALL POSSIBLE SECOND TRANSLOCATIONS:")
    print("-" * 60)
    
    second_step_count = 0
    paths_to_target = []
    
    for intermediate_perm, first_move_list in first_results.items():
        intermediate_str = ''.join(map(str, intermediate_perm))
        
        # Get all possible second moves from this intermediate state
        second_moves = get_all_possible_translocations(intermediate_perm)
        
        print(f"\nFrom intermediate state {intermediate_str}:")
        print(f"(Reachable via {len(first_move_list)} different first moves)")
        
        for j, (second_result, second_move_info) in enumerate(second_moves, 1):
            second_result_str = ''.join(map(str, second_result))
            second_step_count += len(first_move_list)  # Count all combinations
            
            print(f"  {j:2d}. {intermediate_str} → {second_result_str}")
            print(f"      Operation: {second_move_info['description']}")
            
            # Check if this reaches the target
            if second_result == target_perm:
                print(f"      *** REACHES TARGET! ***")
                for first_move_num, first_move_info in first_move_list:
                    paths_to_target.append({
                        'first_move': (first_move_num, first_move_info),
                        'intermediate': intermediate_perm,
                        'second_move': (j, second_move_info),
                        'path': [start_perm, intermediate_perm, second_result]
                    })
    
    print(f"\nTotal possible second translocations: {second_step_count}")
    print(f"2-step paths that reach target: {len(paths_to_target)}")
    
    if paths_to_target:
        print("\n" + "=" * 80)
        print("ALL 2-STEP SOLUTIONS:")
        print("-" * 60)
        
        for i, path_info in enumerate(paths_to_target, 1):
            print(f"\nSOLUTION {i}:")
            path = path_info['path']
            
            print(f"Step 0: {''.join(map(str, path[0]))} (start)")
            print(f"Step 1: {''.join(map(str, path[1]))}")
            print(f"        ← {path_info['first_move'][1]['description']}")
            print(f"Step 2: {''.join(map(str, path[2]))} (target reached!)")
            print(f"        ← {path_info['second_move'][1]['description']}")
    
    return first_results, paths_to_target

def analyze_third_step_from_2step_failures(start_perm, target_perm):
    """
    For states that don't reach target in 2 steps, show what third step options exist.
    """
    print("\n" + "=" * 80)
    print("THIRD STEP ANALYSIS (for 3-step solutions):")
    print("-" * 60)
    
    # Get all possible 2-step states
    first_moves = get_all_possible_translocations(start_perm)
    two_step_states = set()
    two_step_paths = {}  # state -> list of ways to reach it
    
    for result1, move1 in first_moves:
        second_moves = get_all_possible_translocations(result1)
        for result2, move2 in second_moves:
            if result2 != target_perm:  # Only non-target states
                two_step_states.add(result2)
                if result2 not in two_step_paths:
                    two_step_paths[result2] = []
                two_step_paths[result2].append((move1, result1, move2))
    
    print(f"Found {len(two_step_states)} unique states reachable in 2 steps (excluding target)")
    
    three_step_solutions = []
    
    for state in two_step_states:
        state_str = ''.join(map(str, state))
        third_moves = get_all_possible_translocations(state)
        
        # Check which third moves reach the target
        target_reaching_moves = [(move, result) for result, move in third_moves if result == target_perm]
        
        if target_reaching_moves:
            print(f"\nFrom 2-step state {state_str}:")
            print(f"  Can reach target in {len(target_reaching_moves)} ways:")
            
            for move, result in target_reaching_moves:
                print(f"    • {move['description']}")
                
                # Add all complete 3-step paths
                for path_info in two_step_paths[state]:
                    three_step_solutions.append({
                        'step1': path_info[0],
                        'intermediate1': path_info[1], 
                        'step2': path_info[2],
                        'intermediate2': state,
                        'step3': move,
                        'final': result
                    })
    
    print(f"\nTotal 3-step solutions found: {len(three_step_solutions)}")
    
    if three_step_solutions and len(three_step_solutions) <= 20:  # Don't print too many
        print("\nAll 3-step solutions:")
        for i, sol in enumerate(three_step_solutions, 1):
            print(f"\nSOLUTION {i}:")
            print(f"  {start_str} → {''.join(map(str, sol['intermediate1']))} → {''.join(map(str, sol['intermediate2']))} → {target_str}")
            print(f"    Step 1: {sol['step1']['description']}")
            print(f"    Step 2: {sol['step2']['description']}")  
            print(f"    Step 3: {sol['step3']['description']}")
    
    return three_step_solutions

if __name__ == "__main__":
    # Main test case: 1234 → 4321
    start = (4, 3, 2, 1)
    target = (1, 2, 3, 4)
    start_str = ''.join(map(str, start))
    target_str = ''.join(map(str, target))
    
    # Analyze step by step
    first_results, two_step_solutions = analyze_step_by_step_translocations(start, target)
    
    # If no 2-step solutions, analyze 3-step
    if not two_step_solutions:
        three_step_solutions = analyze_third_step_from_2step_failures(start, target)
    
    print("\n" + "=" * 80)
    print("SUMMARY:")
    print(f"• Total possible first translocations: {len(get_all_possible_translocations(start))}")
    print(f"• 2-step solutions: {len(two_step_solutions)}")
    if not two_step_solutions:
        print(f"• 3-step solutions: {len(three_step_solutions) if 'three_step_solutions' in locals() else 'Not calculated'}")
    
    # Also verify with our original comprehensive analysis
    print(f"\nVerification - finding shortest path length:")
    shortest = find_shortest_path_length(start, target)
    print(f"Minimum steps required: {shortest}")

STEP-BY-STEP ANALYSIS: 4321 → 1234
ALL POSSIBLE FIRST TRANSLOCATIONS from 4321:
------------------------------------------------------------
 1. 4321 → 3421
    Operation: Move (4,) from positions [0:1] to position 1

 2. 4321 → 3241
    Operation: Move (4,) from positions [0:1] to position 2

 3. 4321 → 3214
    Operation: Move (4,) from positions [0:1] to position 3

 4. 4321 → 2431
    Operation: Move (4, 3) from positions [0:2] to position 1

 5. 4321 → 2143
    Operation: Move (4, 3) from positions [0:2] to position 2

 6. 4321 → 1432
    Operation: Move (4, 3, 2) from positions [0:3] to position 1

 7. 4321 → 3421
    Operation: Move (3,) from positions [1:2] to position 0

 8. 4321 → 4231
    Operation: Move (3,) from positions [1:2] to position 2

 9. 4321 → 4213
    Operation: Move (3,) from positions [1:2] to position 3

10. 4321 → 3241
    Operation: Move (3, 2) from positions [1:3] to position 0

11. 4321 → 4132
    Operation: Move (3, 2) from positions [1:3] to position 2
